In [64]:
# Data Citation:
# F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on 
# Interactive Intelligent Systems (TiiS) 5, 4: 19:1–19:19. <https://doi.org/10.1145/2827872>

! curl http://files.grouplens.org/datasets/movielens/ml-latest-small.zip -o ml-latest-small.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  1  955k    1 15123    0     0  16493      0  0:00:59 --:--:--  0:00:59 16564
 19  955k   19  189k    0     0   104k      0  0:00:09  0:00:01  0:00:08  104k
 48  955k   48  459k    0     0   161k      0  0:00:05  0:00:02  0:00:03  162k
 72  955k   72  690k    0     0   181k      0  0:00:05  0:00:03  0:00:02  181k
100  955k  100  955k    0     0   200k      0  0:00:04  0:00:04 --:--:--  200k
100  955k  100  955k    0     0   200k      0  0:00:04  0:00:04 --:--:--  244k


In [65]:
import zipfile
with zipfile.ZipFile('ml-latest-small.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

In [66]:
# import the dataset
import pandas as pd
movies_df = pd.read_csv('data/ml-latest-small/movies.csv')
ratings_df = pd.read_csv('data/ml-latest-small/ratings.csv')

In [67]:
print('The dimensions of movies dataframe are:', movies_df.shape,'\nThe dimensions of ratings dataframe are:', ratings_df.shape)

The dimensions of movies dataframe are: (9742, 3) 
The dimensions of ratings dataframe are: (100836, 4)


In [68]:
# Take a look at movies_df
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [69]:
# Take a look at ratings_df
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [70]:
# Movie ID to movie name mapping
movie_names = movies_df.set_index('movieId')['title'].to_dict()
n_users = len(ratings_df.userId.unique())
n_items = len(ratings_df.movieId.unique())
print("Number of unique users:", n_users)
print("Number of unique movies:", n_items)
print("The full rating matrix will have:", n_users*n_items, 'elements.')
print('----------')
print("Number of ratings:", len(ratings_df))
print("Therefore: ", len(ratings_df) / (n_users*n_items) * 100, '% of the matrix is filled.')
print("We have an incredibly sparse matrix to work with here.")
print("And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2")
print("You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.")
print("One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data")

Number of unique users: 610
Number of unique movies: 9724
The full rating matrix will have: 5931640 elements.
----------
Number of ratings: 100836
Therefore:  1.6999683055613624 % of the matrix is filled.
We have an incredibly sparse matrix to work with here.
And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2
You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.
One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data


In [71]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [72]:
import torch
import numpy as np
from torch.autograd import Variable
from tqdm import tqdm_notebook as tqdm

class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        # create user embeddings
        self.user_factors = torch.nn.Embedding(n_users, n_factors) # think of this as a lookup table for the input.
        # create item embeddings
        self.item_factors = torch.nn.Embedding(n_items, n_factors) # think of this as a lookup table for the input.
        self.user_factors.weight.data.uniform_(0, 0.05)
        self.item_factors.weight.data.uniform_(0, 0.05)
        
    def forward(self, data):
        # matrix multiplication
        users, items = data[:,0], data[:,1]
        return (self.user_factors(users)*self.item_factors(items)).sum(1)
    # def forward(self, user, item):
    # 	# matrix multiplication
    #     return (self.user_factors(user)*self.item_factors(item)).sum(1)
    
    def predict(self, user, item):
        return self.forward(user, item)

In [73]:
# Creating the dataloader (necessary for PyTorch)
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader # package that helps transform your data to machine learning readiness

# Note: This isn't 'good' practice, in a MLops sense but we'll roll with this since the data is already loaded in memory.
class Loader(Dataset):
    def __init__(self):
        self.ratings = ratings_df.copy()
        
        # Extract all user IDs and movie IDs
        users = ratings_df.userId.unique()
        movies = ratings_df.movieId.unique()
        
        #--- Producing new continuous IDs for users and movies ---
        
        # Unique values : index
        self.userid2idx = {o:i for i,o in enumerate(users)}
        self.movieid2idx = {o:i for i,o in enumerate(movies)}
        
        # Obtained continuous ID for users and movies
        self.idx2userid = {i:o for o,i in self.userid2idx.items()}
        self.idx2movieid = {i:o for o,i in self.movieid2idx.items()}
        
        # return the id from the indexed values as noted in the lambda function down below.
        self.ratings.movieId = ratings_df.movieId.apply(lambda x: self.movieid2idx[x])
        self.ratings.userId = ratings_df.userId.apply(lambda x: self.userid2idx[x])
        
        
        self.x = self.ratings.drop(['rating', 'timestamp'], axis=1).values
        self.y = self.ratings['rating'].values
        self.x, self.y = torch.tensor(self.x), torch.tensor(self.y) # Transforms the data to tensors (ready for torch models.)

    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.ratings)

In [74]:
num_epochs = 128
cuda = torch.cuda.is_available()

print("Is running on GPU:", cuda)

model = MatrixFactorization(n_users, n_items, n_factors=8)
print(model)
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
# GPU enable if you have a GPU...
if cuda:
    model = model.cuda()

# MSE loss
loss_fn = torch.nn.MSELoss()

# ADAM optimizier
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Train data
train_set = Loader()
train_loader = DataLoader(train_set, 128, shuffle=True)

Is running on GPU: False
MatrixFactorization(
  (user_factors): Embedding(610, 8)
  (item_factors): Embedding(9724, 8)
)
user_factors.weight tensor([[0.0456, 0.0093, 0.0410,  ..., 0.0400, 0.0072, 0.0111],
        [0.0294, 0.0363, 0.0452,  ..., 0.0103, 0.0455, 0.0034],
        [0.0307, 0.0296, 0.0369,  ..., 0.0275, 0.0497, 0.0004],
        ...,
        [0.0116, 0.0078, 0.0012,  ..., 0.0199, 0.0135, 0.0011],
        [0.0239, 0.0135, 0.0333,  ..., 0.0414, 0.0295, 0.0122],
        [0.0415, 0.0101, 0.0234,  ..., 0.0346, 0.0067, 0.0024]])
item_factors.weight tensor([[0.0428, 0.0300, 0.0305,  ..., 0.0443, 0.0102, 0.0013],
        [0.0041, 0.0070, 0.0466,  ..., 0.0487, 0.0039, 0.0325],
        [0.0227, 0.0416, 0.0299,  ..., 0.0050, 0.0298, 0.0407],
        ...,
        [0.0093, 0.0346, 0.0168,  ..., 0.0462, 0.0201, 0.0453],
        [0.0214, 0.0393, 0.0402,  ..., 0.0347, 0.0111, 0.0034],
        [0.0322, 0.0280, 0.0124,  ..., 0.0097, 0.0339, 0.0153]])


In [75]:
losses=[]
for it in range(10):
    if len(losses)>0:
        avg_loss=sum(losses)/len(losses)
    else:
        avg_loss=0
        print("iter #{}".format(it),"Loss:",avg_loss)

iter #0 Loss: 0
iter #1 Loss: 0
iter #2 Loss: 0
iter #3 Loss: 0
iter #4 Loss: 0
iter #5 Loss: 0
iter #6 Loss: 0
iter #7 Loss: 0
iter #8 Loss: 0
iter #9 Loss: 0


In [76]:
# By training the model, we will have tuned latent factors for movies and users.
c = 0
uw = 0
iw = 0 
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
        if c == 0:
          uw = param.data
          c +=1
        else:
          iw = param.data
        #print('param_data', param_data)

user_factors.weight tensor([[0.0456, 0.0093, 0.0410,  ..., 0.0400, 0.0072, 0.0111],
        [0.0294, 0.0363, 0.0452,  ..., 0.0103, 0.0455, 0.0034],
        [0.0307, 0.0296, 0.0369,  ..., 0.0275, 0.0497, 0.0004],
        ...,
        [0.0116, 0.0078, 0.0012,  ..., 0.0199, 0.0135, 0.0011],
        [0.0239, 0.0135, 0.0333,  ..., 0.0414, 0.0295, 0.0122],
        [0.0415, 0.0101, 0.0234,  ..., 0.0346, 0.0067, 0.0024]])
item_factors.weight tensor([[0.0428, 0.0300, 0.0305,  ..., 0.0443, 0.0102, 0.0013],
        [0.0041, 0.0070, 0.0466,  ..., 0.0487, 0.0039, 0.0325],
        [0.0227, 0.0416, 0.0299,  ..., 0.0050, 0.0298, 0.0407],
        ...,
        [0.0093, 0.0346, 0.0168,  ..., 0.0462, 0.0201, 0.0453],
        [0.0214, 0.0393, 0.0402,  ..., 0.0347, 0.0111, 0.0034],
        [0.0322, 0.0280, 0.0124,  ..., 0.0097, 0.0339, 0.0153]])


In [77]:
trained_movie_embeddings = model.item_factors.weight.data.cpu().numpy()

In [78]:
len(trained_movie_embeddings) # unique movie factor weights

9724

In [79]:
from sklearn.cluster import KMeans
# Fit the clusters based on the movie weights
kmeans = KMeans(n_clusters=10, random_state=0).fit(trained_movie_embeddings)

In [80]:
'''It can be seen here that the movies that are in the same cluster tend to have
similar genres. Also note that the algorithm is unfamiliar with the movie name
and only obtained the relationships by looking at the numbers representing how
users have responded to the movie selections.'''
for cluster in range(10):
  print("Cluster #{}".format(cluster))
  movs = []
  for movidx in np.where(kmeans.labels_ == cluster)[0]:
    movid = train_set.idx2movieid[movidx]
    rat_count = ratings_df.loc[ratings_df['movieId']==movid].count()[0]
    movs.append((movie_names[movid], rat_count))
  for mov in sorted(movs, key=lambda tup: tup[1], reverse=True)[:10]:
    print("\t", mov[0])

Cluster #0


C:\Users\gouth\AppData\Local\Temp\ipykernel_10356\1698893749.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  rat_count = ratings_df.loc[ratings_df['movieId']==movid].count()[0]
C:\Users\gouth\AppData\Local\Temp\ipykernel_10356\1698893749.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  rat_count = ratings_df.loc[ratings_df['movieId']==movid].count()[0]
C:\Users\gouth\AppData\Local\Temp\ipykernel_10356\1698893749.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a

	 Forrest Gump (1994)
	 Terminator 2: Judgment Day (1991)
	 Toy Story (1995)
	 American Beauty (1999)
	 Sixth Sense, The (1999)
	 Back to the Future (1985)
	 Mask, The (1994)
	 Die Hard (1988)
	 Inception (2010)
	 Monty Python and the Holy Grail (1975)
Cluster #1


C:\Users\gouth\AppData\Local\Temp\ipykernel_10356\1698893749.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  rat_count = ratings_df.loc[ratings_df['movieId']==movid].count()[0]
C:\Users\gouth\AppData\Local\Temp\ipykernel_10356\1698893749.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  rat_count = ratings_df.loc[ratings_df['movieId']==movid].count()[0]
C:\Users\gouth\AppData\Local\Temp\ipykernel_10356\1698893749.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a

	 Saving Private Ryan (1998)
	 Lord of the Rings: The Two Towers, The (2002)
	 Ace Ventura: Pet Detective (1994)
	 Austin Powers: The Spy Who Shagged Me (1999)
	 Green Mile, The (1999)
	 Being John Malkovich (1999)
	 Iron Man (2008)
	 Jaws (1975)
	 Airplane! (1980)
	 Gattaca (1997)
Cluster #2


...

C:\Users\gouth\AppData\Local\Temp\ipykernel_10356\1698893749.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  rat_count = ratings_df.loc[ratings_df['movieId']==movid].count()[0]
C:\Users\gouth\AppData\Local\Temp\ipykernel_10356\1698893749.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  rat_count = ratings_df.loc[ratings_df['movieId']==movid].count()[0]
C:\Users\gouth\AppData\Local\Temp\ipykernel_10356\1698893749.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a

	 Jurassic Park (1993)
	 Braveheart (1995)
	 Star Wars: Episode V - The Empire Strikes Back (1980)
	 Lion King, The (1994)
	 Gladiator (2000)
	 X-Men (2000)
	 American History X (1998)
	 E.T. the Extra-Terrestrial (1982)
	 Spider-Man (2002)
	 Home Alone (1990)
Cluster #9


C:\Users\gouth\AppData\Local\Temp\ipykernel_10356\1698893749.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  rat_count = ratings_df.loc[ratings_df['movieId']==movid].count()[0]
C:\Users\gouth\AppData\Local\Temp\ipykernel_10356\1698893749.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  rat_count = ratings_df.loc[ratings_df['movieId']==movid].count()[0]
C:\Users\gouth\AppData\Local\Temp\ipykernel_10356\1698893749.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a

	 Star Wars: Episode VI - Return of the Jedi (1983)
	 Fugitive, The (1993)
	 Lord of the Rings: The Return of the King, The (2003)
	 Fargo (1996)
	 Finding Nemo (2003)
	 Indiana Jones and the Last Crusade (1989)
	 Minority Report (2002)
	 Catch Me If You Can (2002)
	 Clear and Present Danger (1994)
	 Departed, The (2006)


C:\Users\gouth\AppData\Local\Temp\ipykernel_10356\1698893749.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  rat_count = ratings_df.loc[ratings_df['movieId']==movid].count()[0]
C:\Users\gouth\AppData\Local\Temp\ipykernel_10356\1698893749.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  rat_count = ratings_df.loc[ratings_df['movieId']==movid].count()[0]
C:\Users\gouth\AppData\Local\Temp\ipykernel_10356\1698893749.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a